# Новая версия модели биэнкодера

In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 24.8 MB/s eta 0:00:00


In [15]:
import os
from datasets import Dataset
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, evaluation, InputExample
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

1. Подготовка данных
Для обучения би-энкодера была проведена переподготовка данных.

Датасет Доктора Хауса увеличен в 3 раза.
Датасет антагонистов расширен до 160 тыс. строк.
Проведены эксперименты по подбору негативных примеров (neg_response)

Финальные метрики косинусного сходства:

anchor ↔ response: 0.2169
anchor ↔ neg_response: 0.1729
response ↔ neg_response: 0.1176


In [ ]:
# === Загрузка данных ===
triplets_df = pd.read_pickle("triplets_limited_cos_extended.pkl")

# === Преобразование данных ===
examples = [
    InputExample(texts=[row["anchor"], row["response"], row["neg_response"]])
    for _, row in triplets_df.iterrows()
]

# === Разделение на train/valid ===
train_examples, valid_examples = train_test_split(examples, test_size=0.2, random_state=42)

2. Выбор модели
Были протестированы несколько архитектур:

sentence-transformers/all-mpnet-base-v2

Высокий raw-результат, но медленное обучение.
Вызывала OOM-ошибки в Google Colab.
BAAI/bge-large-en-v1.5

Сложно поддавалась тюнингу.
Вызывала проблемы с памятью даже при снижении batch_size.
DistilRoBERTa-base (финальный выбор)

Легкая и быстрая модель.
Кастомный пуллинг "mean", который улучшил качество.
Dropout = 0.1, что обеспечило баланс между регуляризацией и переобучением.

3. Подбор гиперпараметров

Размер батча:

16 → обучение нестабильное.
32 → работало, но медленно.
64 → оптимальный вариант.
Learning rate (LR):

2e-5 → модель переобучалась.
5e-6 → стабильное, но слишком медленное обучение.
2e-5 → финальный выбор.
Dropout:

0.1 → переобучение снижено.
0.5 → слишком сильная регуляризация.

Финальные параметры:

batch_size = 64
learning_rate = 2e-5
hidden_dropout_prob = 0.1
pooling_mode = "mean"


In [7]:
# === Установка параметров ===
model_name = "distilroberta-base"
custom_model_path = "./custom_transformer"
batch_size = 64
num_epochs = 30
evaluation_steps = 100

# === Конфигурация модели ===
from transformers import AutoConfig, AutoModel, AutoTokenizer

config = AutoConfig.from_pretrained(model_name)
config.hidden_dropout_prob = 0.1
config.attention_probs_dropout_prob = 0.1

# === Загрузка предобученной модели ===
transformer_model = AutoModel.from_pretrained(model_name, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# === DataLoader ===
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
valid_dataloader = DataLoader(valid_examples, shuffle=False, batch_size=batch_size)

# === Функция потерь ===
train_loss = losses.TripletLoss(model=model)

# === Создание папки для кастомной модели ===
os.makedirs(custom_model_path, exist_ok=True)
transformer_model.save_pretrained(custom_model_path)
tokenizer.save_pretrained(custom_model_path)

# === Подключение к sentence-transformers ===
word_embedding_model = models.Transformer(custom_model_path, max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode="mean")
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

print("Модель успешно загружена с кастомной конфигурацией")

# === Оценка модели перед обучением ===
dev_evaluator = evaluation.TripletEvaluator.from_input_examples(valid_examples, name="dev_evaluator", batch_size=batch_size)
pretrain_score = model.evaluate(dev_evaluator)
print(f"Точность перед обучением: {pretrain_score['dev_evaluator_cosine_accuracy']:.4f}")

# === Запуск обучения ===
print("\nНачинается обучение...")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=int(len(train_dataloader) * num_epochs * 0.05),
    optimizer_params={'lr': 2e-5},
    output_path="./sbert_test"
)

# === Финальная оценка ===
print("\nФинальная оценка модели...")
val_evaluator = evaluation.TripletEvaluator.from_input_examples(valid_examples, name="final_evaluator", batch_size=batch_size)
final_score = model.evaluate(val_evaluator)

print(f"Финальная точность на валидации: {final_score['final_evaluator_cosine_accuracy']:.4f}")


Модель успешно загружена с кастомной конфигурацией
Точность перед обучением: 0.5404

Начинается обучение...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,1.503100
1000,0.230600
1500,0.092300
2000,0.045100
2500,0.023400
3000,0.013300
3500,0.007600
4000,0.007300
4500,0.005000
5000,0.003900



Финальная оценка модели...
Финальная точность на валидации: 0.9775


В процессе тестирования были рассмотрены несколько подходов:

Начальный вариант – использовалась стандартная модель sentence-transformers/all-mpnet-base-v2 без внесения изменений в конфигурацию.
Оптимизация архитектуры – переход на distilroberta-base для уменьшения размера модели и ускорения обучения.

Финальная версия – кастомная конфигурация distilroberta-base с измененными параметрами hidden_dropout_prob=0.2 и attention_probs_dropout_prob=0.2, что позволило снизить переобучение и улучшить устойчивость модели.

In [9]:
# === Сохранение модели ===
model.save("models")
print("Модель сохранена локально в папку models")

Модель сохранена локально в папку models


In [10]:
# Указываем путь для сохранения модели
model_path = "./models/biencoder_model_v2.0"

# Сохраняем модель и токенизатор
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print(f"Модель и токенизатор сохранены в директории {model_path}")

Модель и токенизатор сохранены в директории ./models/biencoder_model_v2.0


In [8]:
import shutil
from google.colab import files

# Упаковка модели в архив ZIP
shutil.make_archive("biencoder_model_v2.0", "zip", "./sbert_test")

# Скачивание архива
files.download("biencoder_model_v2.0.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
from sentence_transformers import SentenceTransformer

# Загрузка модели из папки
model = SentenceTransformer("models/biencoder_model_v2.0")

In [14]:
from huggingface_hub import notebook_login
from sentence_transformers import SentenceTransformer

# Вход в Hugging Face
notebook_login()

# Загружаем модель из сохранённой папки
model_path = "models/biencoder_model_v2.0"
model = SentenceTransformer(model_path)

# Перезаписываем модель в существующем репозитории с новым именем
model.push_to_hub("nikatonika/chatbot_biencoder_v2_cos_sim", commit_message="Update model after training")

print("✅ Модель успешно перезаписана в репозитории chatbot_biencoder на Hugging Face!")


model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

✅ Модель успешно перезаписана в репозитории chatbot_biencoder на Hugging Face!
